In [1]:
trait RNG {
    def nextInt: (Int, RNG)
}

case class SimpleRNG(seed: Long) extends RNG { 
    def nextInt: (Int, RNG) = {
        val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL 
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)
    }
}


defined trait RNG
defined class SimpleRNG


In [2]:
var nowRng = (0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 10) {
    nowRng = nowRng._2.nextInt
    var nextRng = nowRng._2
    println(nowRng._1)
}

384748
-1151252339
-549383847
1612966641
-883454042
1563994289
1331515492
-234691648
672332705
-2039128390


null

In [3]:
// id : ex_6.1

/* Write a function that uses RNG.nextInt to generate a 
   random integer between 0 and Int.maxValue (inclusive). 
   Make sure to handle the corner case when nextInt returns 
   Int.MinValue, which doesn’t have a non-negative counterpart.
*/

// simple idea, map every negative integer to it's postive.
// watching the edge case as indicated
def nextNaturalInt(rng : RNG) : (Int, RNG) = {
    val nextRng = rng.nextInt
    if (nextRng._1 == Int.MinValue) {
        (0, nextRng._2)
    } else {
        (Math.abs(nextRng._1), nextRng._2)
    }
}

nextNaturalInt: (rng: RNG)(Int, RNG)


In [4]:
// id : try ex_6.1

case class RngValue(current : Int) extends RNG {
    def nextInt : (Int, RNG) = {
        (current, this)
    }
}

println(nextNaturalInt(RngValue(Int.MinValue)))
println(nextNaturalInt(RngValue(Int.MinValue+1)))
println(nextNaturalInt(RngValue(-1)))
println(nextNaturalInt(RngValue(0)))
println(nextNaturalInt(RngValue(1)))
println(nextNaturalInt(RngValue(Int.MaxValue-1)))
println(nextNaturalInt(RngValue(Int.MaxValue)))

var nowRng = (0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = nextNaturalInt(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}



(0,RngValue(-2147483648))
(2147483647,RngValue(-2147483647))
(1,RngValue(-1))
(0,RngValue(0))
(1,RngValue(1))
(2147483646,RngValue(2147483646))
(2147483647,RngValue(2147483647))
384748
1151252339
549383847
1612966641
883454042
1563994289
1331515492
234691648
672332705
2039128390
1888584533
294927845
1517050556
92416162
1713389258


null

In [8]:
// id : ex_6.2

def double(rng : RNG) : (Double, RNG) = {
    val nextRngInt = nextNaturalInt(rng)
    (nextRngInt._1.toDouble / (Int.MaxValue.toDouble + 1), nextRngInt._2)
}

double: (rng: RNG)(Double, RNG)


In [9]:
// id : try ex_6.2

println(double(RngValue(0)))
println(double(RngValue(Int.MaxValue)))

var nowRng = (0.0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = double(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}

(0.0,RngValue(0))
(0.9999999995343387,RngValue(2147483647))
1.7916224896907806E-4
0.5360936461947858
0.25582678942009807
0.7510961224325001
0.41139034647494555
0.7282915939576924
0.6200352180749178
0.10928681492805481
0.3130793129093945
0.9495431510731578
0.8794407048262656
0.1373364799655974
0.7064317148178816
0.043034628964960575
0.7978590475395322


null

In [12]:
// id : ex_6.3

def intDouble(rng : RNG) : ((Int, Double), RNG) = {
    val (int, rng1) = nextNaturalInt(rng)
    val (dbl, rng2) = double(rng1)
    ((int, dbl), rng2)
}

def doubleInt(rng : RNG) : ((Double, Int), RNG) = {
    val ((int, dbl), rng1) = intDouble(rng)
    ((dbl, int), rng1)
}

def double3(rng : RNG) : ((Double, Double, Double), RNG) = {
    val (dbl1, rng1) = double(rng)
    val (dbl2, rng2) = double(rng1)
    val (dbl3, rng3) = double(rng2)
    ((dbl1, dbl2, dbl3), rng3)
}

intDouble: (rng: RNG)((Int, Double), RNG)
doubleInt: (rng: RNG)((Double, Int), RNG)
double3: (rng: RNG)((Double, Double, Double), RNG)


In [17]:
// id : try ex_6.3
val rng20 = RngValue(20)
println(intDouble(rng20))
println(doubleInt(rng20))
println(double3(rng20))

((20,9.313225746154785E-9),RngValue(20))
((9.313225746154785E-9,20),RngValue(20))
((9.313225746154785E-9,9.313225746154785E-9,9.313225746154785E-9),RngValue(20))


null

In [56]:
// id : ex_6.4

def ints(count: Int)(rng : RNG) : (List[Int], RNG) = {
    if (count <= 0) {
        (List(), rng)
    } else {
        val (i, rng1) = nextNaturalInt(rng)
        val rem = ints(count-1)(rng1)
        (i::rem._1, rem._2)
    }
}

ints: (count: Int)(rng: RNG)(List[Int], RNG)


In [57]:
// id : try ex_6.4

ints(5)(SimpleRNG(1))

(List(384748, 1151252339, 549383847, 1612966641, 883454042),SimpleRNG(223576932655868))

In [59]:
type Rand[+A] = RNG => (A, RNG)

def map[A,B](s: Rand[A])(f: A => B): Rand[B] = rng => {
    val (a, rng2) = s(rng)
    (f(a), rng2)
}

defined type alias Rand
map: [A, B](s: Rand[A])(f: A => B)Rand[B]


In [66]:
// id : ex_6.5

def double(rng : RNG) : (Double, RNG) =
    map(rng => {
        val (i, r) = nextNaturalInt(rng)
        (i.toDouble, r)
    })(i => i.toDouble / (Int.MaxValue.toDouble + 1))(rng)

double: (rng: RNG)(Double, RNG)


In [67]:
// id : try ex_6.5

println(double(RngValue(0)))
println(double(RngValue(Int.MaxValue)))

var nowRng = (0.0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = double(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}

(0.0,RngValue(0))
(0.9999999995343387,RngValue(2147483647))
1.7916224896907806E-4
0.5360936461947858
0.25582678942009807
0.7510961224325001
0.41139034647494555
0.7282915939576924
0.6200352180749178
0.10928681492805481
0.3130793129093945
0.9495431510731578
0.8794407048262656
0.1373364799655974
0.7064317148178816
0.043034628964960575
0.7978590475395322


null

In [68]:
// id : ex_6.6

def map2[A,B,C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C): Rand[C] = rng => {
    val (a, rng1) = ra(rng)
    val (b, rng2) = rb(rng1)
    (f(a,b), rng2)
}

map2: [A, B, C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C)Rand[C]


In [74]:
// id : try ex_6.6

def both[A,B](ra: Rand[A], rb: Rand[B]): Rand[(A,B)] = map2(ra, rb)((_, _))

val randIntDouble: Rand[(Int, Double)] = both(nextNaturalInt, double)
val randDoubleInt: Rand[(Double, Int)] = both(double, nextNaturalInt)

println(randIntDouble(SimpleRNG(1)))
println(randDoubleInt(SimpleRNG(1)))

((384748,0.5360936461947858),SimpleRNG(206026503483683))
((1.7916224896907806E-4,1151252339),SimpleRNG(206026503483683))


null